In [1]:
import ecole
import shutil
import os
import sys
import time
import numpy as np
from tqdm import trange
import pyscipopt as pyscip
import pickle as pkl

from generate_data import *
from dump_data import *

class HiddenPrints:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout


# Generate Problem and collect features

In [ ]:
scip_parameters = {"branching/scorefunc": "s",
                   "separating/maxrounds": 0,
                   "limits/time": 360,
                   "conflict/enable": False,
                   "presolving/maxrounds": 0,
                   "presolving/maxrestarts": 0,
                   "separating/maxroundsroot" : 0,
                   "separating/maxcutsroot": 0,
                   "separating/maxcuts": 0,
                   "propagating/maxroundsroot" : 0,
                   "lp/presolving" : False,
                  }
generate_dataset(scip_parameters,path = "DataSet/",\
                 nb_cons = [100,200,300,400,500],nb_var = [1,1.5,2],density = [0.1,0.15,0.2])

Generate with Row:100,Col:100,Density:0.100000


 37%|███████████████▌                          | 37/100 [00:04<00:12,  4.95it/s]

pressed CTRL-C 1 times (5 times for forcing termination)


 67%|████████████████████████████▏             | 67/100 [00:08<00:03,  9.46it/s]

# Collect labels

In [5]:

for row in [100,200,300,400,500]:
    for coef_col in [1,1.5,2]: 
        for d in [0.1,0.15,0.2]:
            col = int(coef_col * row)
            print("Solve with Row:%d,Col:%d,Density:%f" % (row,col,d))
            for n in trange(1,101):
#                 #save label
                with HiddenPrints():
                    problme_name = \
                            "set_cover_{"+row.__str__()+"*"+col.__str__()+"_"+d.__str__()+"_"+n.__str__()+"}"
#                     solver = gb.read("problem/"+problme_name+".lp")
#                     solver.optimize()
#                     dumpSolution_Gurobi("DataSet/label/"+problme_name+"_label.json",solver)
                    solver = ecole.scip.Model.from_file("DataSet/"+problme_name+"/problem.lp")
                    aspyscip = solver.as_pyscipopt()
                    aspyscip.setPresolve(pyscip.SCIP_PARAMSETTING.OFF)
                    aspyscip.optimize()
                    gapList.append(aspyscip.getGap())
                    dumpSolution_Ecole("DataSet/"+problme_name+"/label.json",aspyscip)
                    
gap = np.array(gapList)
print(np.mean(gap))
print(np.where(gap != 0))

Solve with Row:100,Col:100,Density:0.100000


100%|█████████████████████████████████████████| 100/100 [00:10<00:00,  9.28it/s]

0.0
(array([], dtype=int64),)


In [52]:
import ast
instance = ecole.scip.Model.from_file("DataSet/set_cover_{100*100_0.1_35}/problem.lp")
aspyscip = instance.as_pyscipopt()
aspyscip.setPresolve(pyscip.SCIP_PARAMSETTING.OFF)
aspyscip.optimize()
bestsol = aspyscip.getBestSol()
print(bestsol)
solutionPool = []
for sol in aspyscip.getSols():
    solutionPool.append(np.array(list(ast.literal_eval(sol.__repr__()).values())).astype(int))
solutions = {
    "Best_Solution":bestsol.__str__(),
    "Solution_Pool":aspyscip.getSols().__str__()
}
print(np.around(list(ast.literal_eval(aspyscip.getSols()[0].__repr__()).values()),0).astype(int).tolist())

{'t_x0': 0.0, 't_x1': 0.0, 't_x2': 1.0, 't_x3': 0.9999999999999999, 't_x4': 1.0, 't_x5': 0.0, 't_x6': 1.0, 't_x7': 1.0, 't_x8': 0.0, 't_x9': 1.0, 't_x10': 1.0, 't_x11': 2.379049338482478e-16, 't_x12': 0.0, 't_x13': 0.0, 't_x14': 1.0, 't_x15': 0.0, 't_x16': 0.0, 't_x17': 0.0, 't_x18': 0.0, 't_x19': 7.477012206659218e-17, 't_x20': 0.0, 't_x21': 0.0, 't_x22': 0.0, 't_x23': 0.0, 't_x24': 0.0, 't_x25': 0.0, 't_x26': 0.0, 't_x27': 0.0, 't_x28': 0.0, 't_x29': 0.0, 't_x30': 0.0, 't_x31': 0.0, 't_x32': 0.0, 't_x33': 0.0, 't_x34': 0.0, 't_x35': 1.0, 't_x36': 1.0, 't_x37': 0.0, 't_x38': 0.0, 't_x39': 0.0, 't_x40': 0.0, 't_x41': 0.0, 't_x42': 0.0, 't_x43': 0.0, 't_x44': 0.0, 't_x45': 0.0, 't_x46': 0.0, 't_x47': 0.0, 't_x48': 1.0, 't_x49': 3.85179416706687e-17, 't_x50': 0.0, 't_x51': 0.0, 't_x52': 9.289621226455392e-17, 't_x53': 0.0, 't_x54': 0.0, 't_x55': 1.8126090197961738e-17, 't_x56': 0.9999999999999998, 't_x57': 0.0, 't_x58': 0.0, 't_x59': 0.0, 't_x60': 0.0, 't_x61': 0.0, 't_x62': 0.0, 't_x63'

In [ ]:
#scip
with HiddenPrints():
    model_scip = pyscip.scip.Model()
    model_scip.setPresolve(pyscip.SCIP_PARAMSETTING.OFF)
    model_scip.readProblem(filename="debugFile/set_cover_test.lp",extension = "lp")
    # model_scip.readProblem(filename="debugFile/debug_diffc.lp",extension = "lp")

model_scip.optimize()
# time.sleep(1)
print("\n",model_scip.getBestSol())

# print("Ecole*********************")
# # with HiddenPrints():
# solver = ecole.scip.Model.from_file("debugFile/debug.lp")
# aspyscip = solver.as_pyscipopt()
# aspyscip.setPresolve(pyscip.SCIP_PARAMSETTING.OFF)
# aspyscip.optimize()
# gap = aspyscip.getGap()
# sol = aspyscip.getBestSol()
# print(sol,gap)

# #gurobi
# print("Gurobi*********************")
# with HiddenPrints():
#     model = gb.read("debugFile/debug.lp")
#     model.optimize()
# for var in model.getVars():
#     print(f"{var.varName}: {round(var.X, 3)}")



In [2]:
import ecole
import shutil
import os
import sys
import time
import numpy as np
from tqdm import trange
import pyscipopt as pyscip
import pickle as pkl

from dump_data import *

scip_parameters = {"branching/scorefunc": "s",
                   "separating/maxrounds": 0,
                   "limits/time": 360,
                   "conflict/enable": False,
                   "presolving/maxrounds": 0,
                   "presolving/maxrestarts": 0,
                   "separating/maxroundsroot" : 0,
                   "separating/maxcutsroot": 0,
                   "separating/maxcuts": 0,
                   "propagating/maxroundsroot" : 0,
                   "lp/presolving" : False,
                  }
done = 0
while not done:
    try:
        setCover = ecole.instance.SetCoverGenerator\
                    (n_rows=10, n_cols=10,density=0.3)
        env = ecole.environment.Branching(observation_function = ecole.observation.NodeBipartite(),\
                                        scip_params = scip_parameters)
        instance = next(setCover)
        #         print(instance)
        problme_name = "set_cover_test"
        instance.write_problem("debugFile/"+problme_name+".lp")

        aspyscip = instance.as_pyscipopt()
        aspyscip.freeTransform()
        obs, action_set, _, _, _ = env.reset(instance)

        print(obs.row_features.shape)
        print(obs.edge_features.shape)
        print(obs.variable_features.shape)
        print(obs.variable_features[0])
        
        done = 1
    except:
        pass

variable = aspyscip.getVars()
print(variable)

(10, 5)
[10, 10]
(10, 20)
[ 0.21156489  1.          0.          0.          0.          1.
  1.         -0.          0.5         0.5         0.          0.
  0.          1.          0.74825175  0.          1.          0.
  0.          8.        ]
[x_0, x_1, x_2, x_3, x_4, x_5, x_6, x_7, x_8, x_9]


In [5]:
original_indice = obs.variable_features[:,-1]
print(original_indice)

[8. 2. 9. 1. 4. 5. 7. 6. 3. 0.]


In [6]:
dumpEdgeFeatures("debugFile/test_EDGEFE.json",obs.edge_features,original_indice)
dumpRowFeatures("debugFile/test_ROWFE.json",obs.row_features)
dumpVariableFeatures("debugFile/test_VarFE.json",obs.variable_features)

In [8]:

aspyscip = instance.as_pyscipopt()
aspyscip.setPresolve(pyscip.SCIP_PARAMSETTING.OFF)
aspyscip.optimize()
dumpSolution_Ecole("debugFile/label.json",aspyscip)

In [86]:
#bias
print(obs.RowFeatures.value)
print(action_set)

[1 3 4 8 9]


In [79]:
edges_dict = {}
for i in range(len(obs.edge_features.indices[0])):
    if obs.edge_features.indices[0][i] not in edges_dict:
        edges_dict[obs.edge_features.indices[0][i]] = []
    edges_dict[obs.edge_features.indices[0][i]].append(obs.edge_features.indices[1][i])
np.set_printoptions(threshold=np.inf)
print(edges_dict)

{0: [3, 4, 6], 1: [2, 5, 7, 9], 2: [0, 1, 2], 3: [0, 3, 6], 4: [0, 5, 7], 5: [2, 8], 6: [0, 5, 9], 7: [0, 1, 4, 7], 8: [3, 5, 8], 9: [4, 5]}


In [128]:
class GetProblemeIndice:

    def __init__(self):
        pass

    def before_reset(self,model):
        pass
    def extract(self, model,done):
        aspyscip = model.as_pyscipopt()
        variable = aspyscip.getVars()
        indices = [var.getObj() for var in variable]
        return indices

In [165]:
scip_parameters = {"branching/scorefunc": "s",
                   "separating/maxrounds": 0,
                   "limits/time": 360,
                   "conflict/enable": False,
                   "presolving/maxrounds": 0,
                   "presolving/maxrestarts": 0,
                   "separating/maxroundsroot" : 0,
                   "separating/maxcutsroot": 0,
                   "separating/maxcuts": 0,
                   "propagating/maxroundsroot" : 0,
                   "lp/presolving" : False,
                   "constraints/linear/sortvars": False,
                   "heuristics/shiftandpropagate/sortvars": False,
                   
                   "propagating/genvbounds/sort": False,
                   "presolving/sparsify/rowsort": False,
                   
                   "randomization/permutevars": False,
                   "randomization/permuteconss": False,
                   
                   "presolving/gateextraction/sorting":0,
                   "separating/oddcycle/sortswitch":0,
                   "separating/oddcycle/sortrootneighbors":False,
                   
#                    "constraints/linear/maxrounds" :0,
#                    "constraints/linear/maxroundsroot" : 0,
#                    "constraints/knapsack/maxroundsroot" : 0,
#                    "separating/maxroundsrootsubrun" : 0,
#                    "separating/zerohalf/maxroundsroot" : 0,
#                    "separating/rlt/maxroundsroot" : 0,
#                    "separating/oddcycle/maxroundsroot" : 0,
#                    "separating/mixing/maxroundsroot" : 0,
#                    "separating/minor/maxroundsroot" : 0,
#                    "separating/interminor/maxroundsroot" : 0,
#                    "separating/gomory/maxroundsroot" : 0,
#                    "separating/eccuts/maxroundsroot" : 0,
#                    "separating/disjunctive/maxroundsroot" : 0,
#                    "separating/aggregation/maxroundsroot" : 0,
                   
#                    "constraints/linear/singletonstuffing": False,
#                    "constraints/linear/rangedrowartcons": False,
#                    "constraints/and/maxprerounds" :0,
#                    "constraints/and/presolusehashing": False,
#                    "constraints/benders/maxprerounds": 0,
#                    "constraints/benderslp/iterlimit": 0,
#                    "reading/dynamicrows": False,
#                    "separator/oddcycle/cutthreshold":0,
                  }
env = ecole.environment.Branching(observation_function = [ecole.observation.NodeBipartite(),GetProblemeIndice()],\
                                scip_params = scip_parameters)
instance = ecole.scip.Model.from_file("debugFile/set_cover_test.lp")
print(instance)

obs, action_set, _, _, _ = env.reset(instance)
get = obs[1]
obs = obs[0]
print(obs.row_features.shape)
print(obs.edge_features.shape)
print(obs.variable_features[:,0])

edges_dict = {}
for i in range(len(obs.edge_features.indices[0])):
    if obs.edge_features.indices[0][i] not in edges_dict:
        edges_dict[obs.edge_features.indices[0][i]] = []
    edges_dict[obs.edge_features.indices[0][i]].append(obs.edge_features.indices[1][i])
np.set_printoptions(threshold=np.inf)
print(edges_dict)
edges = np.zeros((10,10))

print(get)
dumpEdgeFeatures("debugFile/edges_feature.json",obs.edge_features,10,10)

(10, 5)
[10, 10]
[0.32105826 0.36248514 0.15017241 0.14499405 0.07767539 0.41426873
 0.41944709 0.4556956  0.39873365 0.04142687]
{0: [0, 4, 7], 1: [1, 2, 5, 6], 2: [3, 9], 3: [1, 6], 4: [5, 7, 9], 5: [2, 4, 5, 7, 8], 6: [1, 2, 3, 8], 7: [0, 7], 8: [7, 8], 9: [0, 5, 6]}
[28.0, 81.0, 88.0, 15.0, 70.0, 80.0, 8.0, 29.0, 77.0, 62.0]


In [166]:
aspyscip = instance.as_pyscipopt()
aspyscip.setPresolve(pyscip.SCIP_PARAMSETTING.OFF)
aspyscip.optimize()
gapList.append(aspyscip.getGap())
dumpSolution_Ecole("debugFile/label_test.json",aspyscip)

In [124]:
print(env.model)
print(instance)
aspyscip = env.model.as_pyscipopt()
variable = aspyscip.getVars()
for var in variable:
    print(var.getObj() )

28.0
81.0
88.0
15.0
70.0
80.0
8.0
29.0
77.0
62.0


In [ ]:
for row in [100,150,200]:
    for coef_col in [1,1.5,2]: 
        for d in [0.1,0.15,0.2]:
            col = int(coef_col * row)
            print("Solve with Row:%d,Col:%d,Density:%f" % (row,col,d))
            for n in range(1,2):
#                 #save label
                problme_name = \
                        "set_cover_{"+row.__str__()+"*"+col.__str__()+"_"+d.__str__()+"_"+n.__str__()+"}"
#                     solver = gb.read("problem/"+problme_name+".lp")
#                     solver.optimize()
#                     dumpSolution_Gurobi("DataSet/label/"+problme_name+"_label.json",solver)
#                 solver = ecole.scip.Model.from_file("DataSet/problem/"+problme_name+".lp")
#                 aspyscip = solver.as_pyscipopt()
#                 aspyscip.setPresolve(pyscip.SCIP_PARAMSETTING.OFF)
#                 aspyscip.optimize()
                model_scip = pyscip.scip.Model()
                model_scip.setPresolve(pyscip.SCIP_PARAMSETTING.OFF)
                model_scip.readProblem(filename="DataSet/problem/"+problme_name+".lp")
                model_scip.optimize()